# 0. Importing Necessary Packages

In [1]:
# Printing the information of Python, IPython, OS, and the generation date.
# Before running this, you have to install version_information module with "pip install version_information".
%load_ext version_information
%version_information

Software versions
Python 2.7.17 64bit [GCC 7.3.0]
IPython 5.8.0
OS Linux 5.8.18 100.fc31.x86_64 x86_64 with fedora 31 Thirty_One
Sun Apr 24 17:36:36 2022 KST

In [2]:
# Printing the versions of packages
from importlib_metadata import version
for pkg in ['numpy', 'matplotlib', 'pandas', 'astropy', 'pyraf']:
    print(pkg+": ver "+version(pkg))

numpy: ver 1.16.5
matplotlib: ver 2.2.3
pandas: ver 0.24.2
astropy: ver 2.0.16
pyraf: ver 2.1.15


In [3]:
# Matplotlib backend
%matplotlib notebook

# Importing necessary modules
import time
import numpy as np
import glob, os
from matplotlib import pyplot as plt
import pandas as pd
from astropy.io import fits
from astropy.stats import sigma_clipped_stats

current_dir = os.getcwd()    # Current working directory
login_file = "/data/jlee/DATA/TA/2022A/login.cl"    # Please give the absolute path of your 'login.cl' file

# Reading 'login.cl' file
f = open(login_file, "r")
ll = f.readlines()
f.close()

# Finding the string for home directory in 'login.cl' file
line_homedir = np.argwhere(pd.Series(ll).str.startswith("set\thome\t\t=").values)[0][0]
idx_start = ll[line_homedir].find('"')
idx_end = ll[line_homedir].find('"', idx_start+1)
dir_iraf = ll[line_homedir][idx_start+1:idx_end]    # Home directory recoded in the 'login.cl' file
print(dir_iraf)    # For check

# Importing IRAF
os.chdir(dir_iraf)
from pyraf import iraf
os.chdir(current_dir)

/data/jlee/DATA/TA/2022A/


# 1. Displaying the Images

In [6]:
dir_img = "Combined_images/"
imglist = [dir_img+"M105-g.fits", dir_img+"M105-i.fits",
           dir_img+"M100-g.fits", dir_img+"M100-i.fits"]
n_img = len(imglist)
imglist

['Combined_images/M105-g.fits',
 'Combined_images/M105-i.fits',
 'Combined_images/M100-g.fits',
 'Combined_images/M100-i.fits']

In [27]:
# You can also run this command in terminal.
ds9_options = "-scalemode zscale -scale lock yes -frame lock image "
names = ""
for i in np.arange(n_img):
    names += imglist[i]+" "
ds9_command = "ds9 "+ds9_options+names+"&"
print('Running "'+ds9_command+'" in the terminal...')
os.system(ds9_command)

Running "ds9 -scalemode zscale -scale lock yes -frame lock image Combined_images/M105-g.fits Combined_images/M105-i.fits Combined_images/M100-g.fits Combined_images/M100-i.fits &" in the terminal...


0

# 2. Running IRAF/Ellipse Task 

In [8]:
# IRAF/ellipse task is in the STSDAS package (IRAF external package)
iraf.stsdas()
iraf.stsdas.analysis()
iraf.stsdas.analysis.isophote()



      +------------------------------------------------------------+
      |       Space Telescope Science Data Analysis System         |
      |                   STSDAS Version 3.18.3                    |
      |                                                            |
      |   Space Telescope Science Institute, Baltimore, Maryland   |
      |   Copyright (C) 2014 Association of Universities for       |
      |            Research in Astronomy, Inc.(AURA)               |
      |       See stsdas$copyright.stsdas for terms of use.        |
      |         For help, send e-mail to help@stsci.edu            |
      |                                                            |
      +------------------------------------------------------------+
stsdas/:
 analysis/      examples        hst_calib/      sobsolete/
 contrib/       fitsio/         playpen/        toolbox/
 describe       graphics/       problems
isophote/:
 bmodel         geompar@        isomap          magpar@
 contr

In [25]:
# For viewing help file of "ellipse" task
### IRAF.net website has recently stop its service... :(
### But you can still see the detailed description of input parameters of any tasks (but not so convenient) 
iraf.epar("ellipse")
# You have to see the detailed definition of each parameter in geompar, controlpar, samplepar, and magpar.
# iraf.epar("[TASK_NAME]")
### Click "[TASK_NAME] help" for viewing the parameter editor help browser

In [26]:
def fit_ellipse(input_image, output_table=None, interactive=False,
                model_image=None, residual_image=None, display=False, data_file=None,
                x0=100.0, y0=100.0, ellip0=0.1, pa0=45.0, sma0=10.0,  # geompar
                minsma=0.0, maxsma=50.0, step=1.0, linear=False, recenter=False,  # geompar
                minit=10, maxit=100, hcenter=False, hellip=False, hpa=False,  # controlpar
                usclip=3.0, lsclip=3.0, nclip=0,  # samplepar
                mag0=25.0,  # magpar
                backgr=0.0, interp='nearest'):  # bmodel

    '''
    # --- basic input parameters --- #
    input_image - input image name ('[FILENAME].fits')
    output_table - output table name (default: '[FILENAME].tab')
    interactive - interactive (boolean, default: False)
    model_image - output model image (default: '[FILENAME]_mod.fits')
    residual_image - output residual image (default: '[FILENAME]_res.fits')
    data_file - output data file (default: '[FILENAME].dat')
    display - display the results or not? (boolean, default: False)
    
    # --- geompar set --- #
    x0, y0 - initial isophote center X, Y [pixel]
    ellip0, pa0 - initial ellipticity, position angle [degree]
    sma0 - initial semi-major axis (SMA) length [pixel]
    minsma - minimum SMA length for fitting [pixel] (default: 0.0)
    maxsma - maximum SMA length for fitting [pixel]
    step - SMA step between successive ellipses [pixel]
    linear - linear SMA step for fitting? (boolean, default: False)
    recenter - do you allow to re-center x0 & y0? (boolean, default: False)
    
    # --- controlpar set --- #
    minit - minimum iteration number at each step of SMA (default: 10)
    maxit - maximum iteration number at each step of SMA (default: 100)
    hcenter - do you want to hold center fixed? (boolean, default: False)
    hellip - do you want to hold ellipticity fixed? (boolean, default: False)
    hpa - do you want to hold position angle fixed? (boolean, default: False)
    
    # --- samplepar set --- #
    usclip - upper sigma-clip criterion (default: 3)
    lsclip - lower sigma-clip criterion (default: 3)
    nclip - iteration number for the sigma clipping (default: 0)
    
    # --- magpar set --- #
    mag0 - magnitude zeropoint (default: 25.0)
    
    # --- bmodel parameter set --- #
    backgr - background level for making model image (default: 0.0)
    interp - interpolation algorithm for model image ('nearest' OR 'linear' OR'poly3' OR 'spline', default: 'nearest')    
    '''
    
    iname = input_image.split('.fits')[0].split('/')[-1]    # Image name
    if (output_table is None):
        output_table = iname+'.tab'    # Output table name
    if (model_image is None):
        model_image = iname+'_mod.fits'    # Output model image name
    if (residual_image is None):
        residual_image = iname+'_res.fits'    # Output residual image name
    if (data_file is None):
        data_file = iname+'.dat'    # Output data file name 

    # Reset by removing the output data
    os.system('rm -rfv '+output_table+' colnames.lis '+iname+'.dat')
    
    # Running IRAF/ellipse task
    kwargs = {"x0":x0, "y0":y0, "ellip0":ellip0, "pa0":pa0, "sma0":sma0,
              "minsma":minsma, "maxsma":maxsma, "step":step, "linear":"no", "recenter":"no",
              "minit":minit, "maxit":maxit, "hcenter":"no", "hellip":"no", "hpa":"no",
              "integrmode":"bi-linear", "usclip":usclip, "lsclip":lsclip, "nclip":nclip,
              "mag0":mag0, "refer":1.0E-5, "zerolevel":0.0}
    iraf.ellipse(input=input_image, output=output_table, interactive=interactive,
                 **kwargs)
    
    # Making model, residual images
    iraf.bmodel(table=output_table, output=model_image, parent=input_image,
                backgr=backgr, interp=interp)    # bmodel task for model image
    
    iraf.imarith(input_image, "-", model_image, residual_image)    # input - model = residual
    
    if display:    # if display == True, DS9 will display input, model, and residual images.
        ds9_options += " -tile grid manual -tile grid layout 3 1 "
        os.system("ds9 "+ds9_options+input_image+" "+residual_image+"&")
    
    # Reading output results
    ### output_table is not directly readable because it is a binary-format file :( 
    iraf.tlcol(output_table, nlist=1, Stdout='colnames.lis')    # Extracting column names
    iraf.tdump(table=output_table, columns="@colnames.lis", datafile=data_file)    # Making ASCII data file
    
    f = open("colnames.lis", "r")
    cc = f.readlines()
    f.close()

    colnames = []
    for line in cc:
        if not (line[0] == '#'):
            colnames.append(line.split(' ')[0])
#     print(colnames)    # column names array
    
    iso_tbl = np.genfromtxt(output_table, encoding="ascii", names=colnames)    # Reading data
    print(iso_tbl)
    
    return iso_tbl

In [28]:
# For g-band image of M105
imgname = "Combined_images/M105-g.fits"
x_center, y_center = 455.0, 455.0    # depending on your object
r0 = 420.0    # maximum SMA

# --- Background estimation for determining backgroun level --- #
### (This is up to you! You do not have to do this if the background level in your images can be obviously determined.)
img = fits.getdata(imgname, ext=0)

x1d = np.arange(0, img.shape[1], 1)
y1d = np.arange(0, img.shape[0], 1)
xx, yy = np.meshgrid(x1d, y1d, sparse=True)
z = ((xx-x_center)**2.0 + (yy-y_center)**2.0 - r0**2.0)
sky_region = (z > 0.0)

avg, med, std = sigma_clipped_stats(img[sky_region], sigma=3.0)
sky_val, sky_sig = 3.0*med - 2.0*avg, std
print(sky_val, sky_sig)
# ---------- #

# kwargs = {"input_image":imgname, "display":True,
#           "x0":455.0, "y0":455.0, "ellip0":0.05, "sma0":50.0,
#           "maxsma":420, "step":0.1,
#           "hcenter":False, "hellip":False, "hpa":False,
#           "nclip":1,
#           "mag0":22.5}  # Here you can change input (default) parameter if needed!


# _ = fit_ellipse()


# def fit_ellipse(input_image, output_table=None, interactive=False,
#                 model_image=None, residual_image=None, display=False, data_file=None,
#                 x0=100.0, y0=100.0, ellip0=0.1, pa0=45.0, sma0=10.0,  # geompar
#                 minsma=0.0, maxsma=50.0, step=1.0, linear=False, recenter=False,  # geompar
#                 minit=10, maxit=100, hcenter=False, hellip=False, hpa=False,  # controlpar
#                 usclip=3.0, lsclip=3.0, nclip=0,  # samplepar
#                 mag0=25.0,  # magpar
#                 backgr=0.0, interp='nearest'

(0.02016184872497758, 0.015442665365112712)


In [ ]:
def fit_ellipse(input_image, initial_guess_region, step=0.05, maxsma=100.,
                plot_isophot=True, imgdata=None, n_isophot=4,
                origin='lower', cmap='gray_r', vmin=-0.01, vmax=0.05,
                plot_profile=True, boundary_value=None):
    '''
    input_image - the input image of the masked data array (dtype: string)
    initial_guess_region - the initial ellipse region from DS9 (dtype: string)
    step - step for ellipse fitting (dtype: numpy.float)
    maxsma - maximum semi-major axis for ellipse fitting (dtype: numpy.float)
    plot_isophot - if True, plot the isophotal lines over the image data (default: True)
    imgdata - the original image data array (dtype: np.array)
    n_isophot - the number of isophot lines to plot (dtype: numpy.int)
    origin, cmap, vmin, vmax - matplotlib plot parameters
    plot_profile - if True, plot the intensity profile (default: True)
    boundary_value - the galaxy & tail boundary value (dtype: numpy.float)
    '''
    x0, y0, a, b, pa = read_region(initial_guess_region, regtype='ellipse')
    x0, y0 = x0[0], y0[0]
    sma, eps = a[0], b[0]/a[0]
    if ((pa[0]-90.) % 180 > 90.):
        pa = (pa[0]-90.) % 180 - 180.
    elif ((pa[0]-90.) % 180 == 0):
        pa = 1./3600
    else:
        pa = (pa[0]-90.) % 180
  
    iname = input_image.split('.fits')[0]

    os.system('rm -rfv '+iname+'.tab colnames.lis '+iname+'.dat')
    
    iraf.ellipse(input=iname+'.fits', output=iname+'.tab', interac='no',
                 x0=x0, y0=y0, ellip0=eps, pa0=pa, sma0=sma, minsma=0.0, maxsma=maxsma, step=step,
                 linear='no', recenter='no', mag0=25.0, refer=1.0E-5, zerolev=0.0, maxit=100,
                 hcenter='no', hellip='no', hpa='no')

    iraf.tlcol(iname+'.tab', nlist=1, Stdout='colnames.lis')
    iraf.tdump(table=iname+'.tab', columns="@colnames.lis", datafile=iname+'.dat')
    
    f = open("colnames.lis", "r")
    cc = f.readlines()
    f.close()

    colnames = []
    for line in cc:
        if not (line[0] == '#'):
            colnames.append(line.split(' ')[0])
    print(colnames)
    
    iso_tbl = np.genfromtxt(iname+".dat", encoding="ascii", names=colnames)

    if plot_isophot:
        fig, ax = plt.subplots()
        ax.imshow(imgdata, origin=origin, cmap=cmap, vmin=vmin, vmax=vmax)

        smas = np.linspace(0.2*maxsma, 0.8*maxsma, 4)
        for sma in smas:
            idx = np.abs(iso_tbl['SMA'] - sma).argmin()
            e = ell((iso_tbl['X0'][idx], iso_tbl['Y0'][idx]),
                    width=2*iso_tbl['SMA'][idx],
                    height=2*iso_tbl['SMA'][idx]*(1-iso_tbl['ELLIP'][idx]),
                    angle=iso_tbl['PA'][idx]+90.,
                    fill=False, color='red', linestyle='-', linewidth=1.25)
            ax.add_patch(e)
    
    if plot_profile:
        fig, ax = plt.subplots()
        ax.set_xlabel(r"${\rm SMA}^{1/4}$")
        ax.set_ylabel("Log (Intensity)")
        ax.set_xlim([0.0, maxsma**0.25])
        ymin = np.log10(iso_tbl['INTENS'][iso_tbl['INTENS'] > 0.].min()*0.5)
        ymax = np.log10(iso_tbl['INTENS'][iso_tbl['INTENS'] > 0.].max()/0.5)
        ax.set_ylim([ymin, ymax])
        ax.plot(iso_tbl['SMA']**0.25, np.log10(iso_tbl['INTENS']), 'o', ms=3.0)
        ax.axhline(np.log10(boundary_value), 0.0, maxsma**0.25,
                   linestyle='-', linewidth=1.5, color='gray', alpha=0.9)
        plt.tight_layout()
    
    return iso_tbl